- Cargar imagen y librerías

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

imagen = cv2.imread('imagen_reto_convolusiones_filtrado.jpg', cv2.IMREAD_COLOR)

cv2_imshow(imagen)

- Función para agregar ruido Salt&Pepper

In [ ]:
def salt_pepper(imagen, prob):
  ruido = imagen.copy()
  ruido = cv2.cvtColor(ruido, cv2.COLOR_BGR2GRAY)

  rand = np.random.rand(*imagen.shape[:2])
  ruido[rand < prob/2] = 0
  ruido[rand > 1 - prob/2] = 255

  return ruido

imagen_ruido = salt_pepper(imagen, 0.1)

cv2_imshow(imagen_ruido)

- Diccionario de kernels

In [ ]:
import numpy as np

kernels = {
  "media": np.ones((3,3), dtype=np.float32) / 9,

  "media5": np.ones((5,5), dtype=np.float32) / 25,

  "gaussiano": np.array([
      [1, 2,  1],
      [2, 4,  2],
      [1, 2,  1]
  ], dtype=np.float32) / 16,

  "gaussiano5": np.array([
      [1, 4,  6,  4,  1],
      [4, 16, 24, 16, 4],
      [6, 24, 36, 24, 6],
      [4, 16, 24, 16, 4],
      [1, 4,  6,  4,  1]
  ], dtype=np.float32) / 256,

  "box_pesado": np.array([
      [1, 1,  1],
      [1, 2,  1],
      [1, 1, 1]
  ], dtype=np.float32) / 10
}

- Función para hacer convolusion con un kernel específico y comparar con la función de OpenCV

In [ ]:
import numpy as np

def convolucion(imagen, kernel):
    alto, ancho = imagen.shape
    k = kernel.shape[0] // 2

    salida = np.zeros_like(imagen, dtype=np.float32)

    for y in range(k, alto - k):
        for x in range(k, ancho - k):

            suma = 0.0
            for i in range(-k, k + 1):
                for j in range(-k, k + 1):
                    suma += imagen[y + i, x + j] * kernel[i + k, j + k]

            salida[y, x] = suma

    return salida

kernel = kernels["box_pesado"]
img_convolusionada = convolucion(imagen_ruido, kernel)
img_convolusionada = np.clip(img_convolusionada, 0, 255).astype(np.uint8)
print("Sin OpenCV")
cv2_imshow(img_convolusionada)

comparar_funcion = cv2.filter2D(imagen_ruido, -1, kernel)
print("Con OpenCV")
cv2_imshow(comparar_funcion)

- Funciones propias de OpenCV

In [ ]:
img_media = cv2.blur(imagen_ruido, (3,3))
cv2_imshow(img_media)

img_media5 = cv2.medianBlur(imagen_ruido, 5)
cv2_imshow(img_media5)

img_gaussiano = cv2.GaussianBlur(imagen_ruido, (3,3), 0)
cv2_imshow(img_gaussiano)

img_gaussiano5 = cv2.GaussianBlur(imagen_ruido, (5,5), 0)
cv2_imshow(img_gaussiano5)

### Analisis

- Aproxidamente a partir de la probabilidad 0.7 teniendo previo conocimiento de la imagen, ya no se distingue nada de la imagen original
- Una vez que se le aplica ruido a una imagen, esta ya no vuelve a ser igual, porque incluso si se elimina dicho ruido va a quedar distorsionada
- Se observa que entre los kernels 3x3 no hay diferencia suficente para decir cual de ellos es mejor, aún queda la imagen con ruido y se ve un poco difuminada, y los kernels 5x5 eliminan bastante el ruido pero a su vez difuminan mucho más la imagen.
- La funcion propia y la funcion filter2D realmente no tienen mucha diferencia al momento de aplicar los diferentes kernels
- Se emplearon distintos filtros disponibles en OpenCV, destacando el filtro de media 5x5 como el más efectivo para ruido salt & pepper